[참고] : https://towardsdatascience.com/image-captioning-with-keras-teaching-computers-to-describe-pictures-c88a46a311b8

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import string
import os
from PIL import Image
from time import time
from keras.layers import Dense, Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.utils import img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras
from keras.models import Model
from keras import Input, layers
from tqdm import tqdm

In [3]:
# 압축파일 압축해제 코드
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/5조/데이터/정리 전 데이터/음식.zip', 'r') as zip:
    zip.extractall('./temp1')

with ZipFile('/content/drive/MyDrive/5조/데이터/정리 전 데이터/반려동물_완.zip', 'r') as zip:
    zip.extractall('./temp2')

with ZipFile('/content/drive/MyDrive/5조/데이터/정리 전 데이터/육아.zip', 'r') as zip:
    zip.extractall('./temp3')

with ZipFile('/content/drive/MyDrive/5조/데이터/정리 전 데이터/디저트.zip', 'r') as zip:
    zip.extractall('./temp4')

with ZipFile('/content/drive/MyDrive/5조/데이터/정리 전 데이터/네일아트_완.zip', 'r') as zip:
    zip.extractall('./temp5')

with ZipFile('/content/drive/MyDrive/5조/데이터/정리 전 데이터/옷스타그램 (재업).zip', 'r') as zip:
    zip.extractall('./temp6')

with ZipFile('/content/drive/MyDrive/5조/데이터/정리 전 데이터/커피(이름 수정).zip', 'r') as zip:
    zip.extractall('./temp7')

with ZipFile('/content/drive/MyDrive/5조/데이터/정리 전 데이터/맛스타그램.zip', 'r') as zip:
    zip.extractall('./temp8')

with ZipFile('/content/drive/MyDrive/5조/데이터/정리 전 데이터/꽃.zip', 'r') as zip:
    zip.extractall('./temp9')

In [4]:
columns = ['이미지id', '음식', '음료수', '성인', '아동', '동물', '네일', '꽃', '기타', '패션', '쓰레기']
df_total = pd.read_csv('/content/drive/MyDrive/5조/모델/07.21 객체검출 모델/학습데이터.csv')

In [5]:
# 각 폴더에 있는 파일 맨 앞값과 끝값 확인(필요할 때 사용)
image_path1='/content/temp1/'
image_path2='/content/temp2/╣▌╖┴╡┐╣░/'
image_path3='/content/temp3/└░╛╞/'
image_path4='/content/temp4/'
image_path5='/content/temp5/'
image_path6='/content/temp6/'
image_path7='/content/temp7/'
image_path8='/content/temp8/'
image_path9='/content/temp9/▓╔/'

image_data=os.listdir(image_path9)
lst=[]
for data in image_data:
  lst.append(int(data.split(".")[0]))
lst.sort()
lst[0], lst[-1]

(4000000, 4009458)

In [6]:
# 이미지의 벡터화를 진행하는 코드

# Load the inception v3 model
model_v3 = InceptionV3(weights='imagenet')

# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model_v3.input, model_v3.layers[-2].output)


96124928/96112376 [==============================] - 3s 0us/step


In [ ]:
# X_train.shape
# (50000, 32, 32, 3)

(50000, 32, 32, 3)

In [7]:
# Function to encode a given image into a vector of size (2048, )
def encode(image_path):
    img = keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    # Convert image to numpy array of 3-dimensions
    x = img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    fea_vec = model_new.predict(x) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec   # 이 데이터가 input data로 사용됨

In [8]:
# data=[]
# for image in tqdm(df_total['이미지id'].values ):
#   if int(image[0])==1:
#     path=image_path1+image
#     data.append(np.array(encode(path)))
#   elif int(image[0])==2:
#     path=image_path2+image
#     data.append(np.array(encode(path)))
#   elif int(image[0])==3:
#     path=image_path3+image
#     data.append(np.array(encode(path)))
#   elif int(image[0])==6:
#     if int(image.split('.')[0])>=6028883 and int(image.split('.')[0])<= 6047176:
#       path=image_path4+image
#       data.append(np.array(encode(path)))
#     elif int(image.split('.')[0])>=6000000 and int(image.split('.')[0])<= 6007780:
#       path=image_path5+image
#       data.append(np.array(encode(path)))
#     elif int(image.split('.')[0])>=6007781 and int(image.split('.')[0])<= 6028882:
#       path=image_path6+image
#       data.append(np.array(encode(path)))
#     elif int(image.split('.')[0])>=6056666 and int(image.split('.')[0])<= 6073230:
#       path=image_path7+image
#       data.append(np.array(encode(path)))
#     elif int(image.split('.')[0])>=6047177 and int(image.split('.')[0])<= 6056665:
#       path=image_path8+image
#       data.append(np.array(encode(path)))
#   elif int(image[0])==4:
#     path=image_path9+image
#     data.append(np.array(encode(path)))

data = np.load('/content/drive/MyDrive/5조/모델/07.21 객체검출 모델/training_data.npy')

In [9]:
# 중복 데이터 제거 코드
dic = {}
for index, i in enumerate(data):
  dic[tuple(i)] = index

new_data = []
for i in dic.values():
  new_data.append(np.array(data[i]))
new_data = np.array(new_data)

In [10]:
data = []
for image in tqdm(df_total['이미지id']):
  if int(image[0])==1:
    path=image_path1+image
    img = keras.preprocessing.image.load_img(path, target_size=(299, 299))
    data.append(np.array(img))
  elif int(image[0])==2:
    path=image_path2+image
    img = keras.preprocessing.image.load_img(path, target_size=(299, 299))
    data.append(np.array(img))
  elif int(image[0])==3:
    path=image_path3+image
    img = keras.preprocessing.image.load_img(path, target_size=(299, 299))
    data.append(np.array(img))
  elif int(image[0])==6:
    if int(image.split('.')[0])>=6028883 and int(image.split('.')[0])<= 6047176:
      path=image_path4+image
      img = keras.preprocessing.image.load_img(path, target_size=(299, 299))
      data.append(np.array(img))
    elif int(image.split('.')[0])>=6000000 and int(image.split('.')[0])<= 6007780:
      path=image_path5+image
      img = keras.preprocessing.image.load_img(path, target_size=(299, 299))
      data.append(np.array(img))
    elif int(image.split('.')[0])>=6007781 and int(image.split('.')[0])<= 6028882:
      path=image_path6+image
      img = keras.preprocessing.image.load_img(path, target_size=(299, 299))
      data.append(np.array(img))
    elif int(image.split('.')[0])>=6056666 and int(image.split('.')[0])<= 6073230:
      path=image_path7+image
      img = keras.preprocessing.image.load_img(path, target_size=(299, 299))
      data.append(np.array(img))
    elif int(image.split('.')[0])>=6047177 and int(image.split('.')[0])<= 6056665:
      path=image_path8+image
      img = keras.preprocessing.image.load_img(path, target_size=(299, 299))
      data.append(np.array(img))
  elif int(image[0])==4:
    path=image_path9+image
    img = keras.preprocessing.image.load_img(path, target_size=(299, 299))
    data.append(np.array(img))

100%|██████████| 30767/30767 [18:35<00:00, 27.57it/s]


In [11]:
new_data = []
for index, i in enumerate(data):
  if index in dic.values():
    new_data.append(i)

In [12]:
np.save('before_aug_data', new_data) 

In [13]:
Y=df_total.iloc[list(dic.values()), 1:].values

In [14]:
X_train, X_test, Y_train, Y_test=train_test_split(
    new_data, Y, test_size=0.2, random_state=2022
)

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,          # randomly rotate images in the range 15 degree
    width_shift_range=0.1,      # randomly shift images horizontally 
    height_shift_range=0.1,     # randomly shift images vertically
    shear_range=0.1,            # 도형의 기울기
    zoom_range=[0.8,1.2],       # 0.8~1.2 배 크기로 임의로 확대/축소
    horizontal_flip=True,       # randomly flip images
    vertical_flip=False,        # randomly flip images
    fill_mode='nearest'         # set mode for filling points outside the input boundaries
)

In [ ]:
datagen.fit(X_train)

In [ ]:
# 모델 정의
inputs1 = Input(shape=(2048,))
fe1 = Dense(256, activation='relu')(inputs1)
fe2 = Dense(512, activation='relu')(fe1)
decoder1 = Dense(256, activation='relu')(fe2)
outputs = Dense(8, activation='sigmoid')(decoder1)   # multi label classification 모델이므로 activation이 sigmoid가 되어야 함
model = Model(inputs=inputs1, outputs=outputs)

# 모델 설정
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_path='best_model.h5'
mc=ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es=EarlyStopping(patience=10)

# 모델 훈련
hist=model.fit(
    X_train, Y_train, validation_split=0.2, epochs=100, callbacks=[mc,es],  batch_size=64
)

Epoch 1/100
295/299 [============================>.] - ETA: 0s - loss: 0.2248 - accuracy: 0.6842
Epoch 1: val_loss improved from inf to 0.20632, saving model to best_model.h5
299/299 [==============================] - 2s 5ms/step - loss: 0.2246 - accuracy: 0.6846 - val_loss: 0.2063 - val_accuracy: 0.7054
Epoch 2/100
298/299 [============================>.] - ETA: 0s - loss: 0.1914 - accuracy: 0.7309
Epoch 2: val_loss did not improve from 0.20632
299/299 [==============================] - 1s 4ms/step - loss: 0.1914 - accuracy: 0.7308 - val_loss: 0.2070 - val_accuracy: 0.7084
Epoch 3/100
293/299 [============================>.] - ETA: 0s - loss: 0.1828 - accuracy: 0.7393
Epoch 3: val_loss improved from 0.20632 to 0.19818, saving model to best_model.h5
299/299 [==============================] - 1s 4ms/step - loss: 0.1831 - accuracy: 0.7393 - val_loss: 0.1982 - val_accuracy: 0.7324
Epoch 4/100
298/299 [============================>.] - ETA: 0s - loss: 0.1766 - accuracy: 0.7492
Epoch 4: val

In [ ]:
from sklearn.metrics import accuracy_score

best_model=load_model('best_model.h5')
label=best_model.predict(X_test)

y_pred=[]
for sample in label:
  y_pred.append([1 if i>=0.5 else 0 for i in sample ] )
y_pred = np.array(y_pred)

print('total_score', accuracy_score(Y_test, y_pred))

real_df=pd.DataFrame(Y_test)
pred_df=pd.DataFrame(y_pred)

for i in range(len(real_df.columns)):
  print(columns[1:][i], accuracy_score( real_df[i].values, pred_df[i].values ) )

total_score 0.6396652719665272
음식 0.9010878661087867
음료수 0.916652719665272
성인 0.8938912133891214
아동 0.9384100418410042
동물 0.9626778242677825
네일 0.9688702928870293
꽃 0.9327196652719665
기타 0.938744769874477


In [ ]:
# 중복제거
# total_score 0.7979916317991632
# 음식 0.9579916317991631
# 음료수 0.9554811715481172
# 성인 0.9422594142259414
# 아동 0.9589958158995816
# 동물 0.9866108786610879
# 네일 0.9882845188284519
# 꽃 0.9638493723849373
# 기타 0.9511297071129707

In [ ]:
np.save('training_data', new_data) 

In [ ]:
df_total.to_csv('학습데이터.csv', index=False)